# Minimal chromospyce example

In [14]:
import random
import chromospyce
import numpy as np
import pyarrow as pa

BINS_NUM = 1000

def make_random_3D_chromatin_structure(n):
    position = np.array([0.0, 0.0, 0.0])
    positions = [position.copy()]
    x_arr = []
    y_arr = []
    z_arr = []
    for _ in range(n):
        step = np.random.choice([-1.0, 0.0, 1.0], size=3)  # Randomly choose to move left, right, up, down, forward, or backward
        position += step
        positions.append(position.copy())
        x_arr.append(position[0])
        y_arr.append(position[1])
        z_arr.append(position[2])
    return [x_arr, y_arr, z_arr]

# Step 1: Generate random structure and convert from arrays to 
#         Apache Arrow table with 3 columns: 'x', 'y', 'z'
random_structure = make_random_3D_chromatin_structure(BINS_NUM)
table = pa.Table.from_arrays(random_structure, names=["x", "y", "z"])

# Convert the Table to bytes
output_stream = pa.BufferOutputStream()
with pa.ipc.RecordBatchStreamWriter(output_stream, table.schema) as writer:
    writer.write_table(table)

table_bytes = output_stream.getvalue().to_pybytes()

# Step 2: Display the structure in a chromospyce widget
numbers = list(range(1, BINS_NUM+1))
vc = {
    # "color": "#6600ff", 
    "color": {
        "values": numbers,
        "min": 1,
        "max": BINS_NUM,
        "colorScale": "Spectral"
    }, 
    "scale": 0.01, 
    "links": True, 
    "mark": "sphere"
}
chromospyce.Widget(structure=table_bytes, viewconfig=vc)

Widget(structure=b'\xff\xff\xff\xff\xc8\x00\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0c\x00\x06\x00\x05\x00\x08\…